In [1]:
import numpy as np
import pandas as pd
import random
from sklearn import preprocessing
from collections import deque

In [2]:
df = pd.read_csv("C:/Users/H_PC/Desktop/Deep Learning/practice/crypto_data/LTC-USD.csv", names= ["time", "low", "high", "open", "close", "volume"])
df.head()

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978


In [3]:
main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"] 

for ratio in ratios:
    print(ratio)
    
    dataset = f"C:/Users/H_PC/Desktop/Deep Learning/practice/crypto_data/{ratio}.csv"
    
    df = pd.read_csv(dataset, names= ["time", "low", "high", "open", "close", "volume"])
    
    df.rename(columns= {"close": f"{ratio}_close",
                        "volume": f"{ratio}_volume"},
             inplace= True)
    
    df.set_index("time", inplace= True)
    
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

main_df.fillna(method= "ffill", inplace= True)
main_df.dropna(inplace= True)
main_df.head()

BTC-USD
LTC-USD
BCH-USD
ETH-USD


,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume
time,,,,,,,,
1528968720,6487.379883,7.706374,96.660004,314.387024,870.859985,26.856577,486.01001,26.019083
1528968780,6479.410156,3.088252,96.570000,77.129799,870.099976,1.124300,486.00000,8.449400
1528968840,6479.410156,1.404100,96.500000,7.216067,870.789978,1.749862,485.75000,26.994646
1528968900,6479.979980,0.753000,96.389999,524.539978,870.000000,1.680500,486.00000,77.355759
1528968960,6480.000000,1.490900,96.519997,16.991997,869.989990,1.669014,486.00000,7.503300


In [4]:
Sequence_Length = 60
Future_Period_Predict = 3
Ratio_to_Predict = "LTC-USD"

In [5]:
main_df["future"] = main_df[f"{Ratio_to_Predict}_close"].shift(-Future_Period_Predict)
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume,future
time,,,,,,,,,
1528968720,6487.379883,7.706374,96.660004,314.387024,870.859985,26.856577,486.01001,26.019083,96.389999
1528968780,6479.410156,3.088252,96.570000,77.129799,870.099976,1.124300,486.00000,8.449400,96.519997
1528968840,6479.410156,1.404100,96.500000,7.216067,870.789978,1.749862,485.75000,26.994646,96.440002
1528968900,6479.979980,0.753000,96.389999,524.539978,870.000000,1.680500,486.00000,77.355759,96.470001
1528968960,6480.000000,1.490900,96.519997,16.991997,869.989990,1.669014,486.00000,7.503300,96.400002


In [6]:
def Classify (current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [7]:
main_df["target"] = list(map(Classify, main_df[f"{Ratio_to_Predict}_close"], main_df["future"]))
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume,future,target
time,,,,,,,,,,
1528968720,6487.379883,7.706374,96.660004,314.387024,870.859985,26.856577,486.01001,26.019083,96.389999,0
1528968780,6479.410156,3.088252,96.570000,77.129799,870.099976,1.124300,486.00000,8.449400,96.519997,0
1528968840,6479.410156,1.404100,96.500000,7.216067,870.789978,1.749862,485.75000,26.994646,96.440002,0
1528968900,6479.979980,0.753000,96.389999,524.539978,870.000000,1.680500,486.00000,77.355759,96.470001,1
1528968960,6480.000000,1.490900,96.519997,16.991997,869.989990,1.669014,486.00000,7.503300,96.400002,0


In [8]:
times = sorted(main_df.index.values)

last_5pct = sorted(main_df.index.values)[-int(0.05 * len(times))]

validation_main_df = main_df[main_df.index > last_5pct]

main_df = main_df[(main_df.index < last_5pct)]

In [9]:
def Preprocessing_df(df):
    
    # df = df.drop("future", 1)
    
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace= True)
            df[col] = preprocessing.scale(df[col].values)

    df.dropna(inplace= True)
    
    Sequential_data = []
    
    prev_days = deque(maxlen= Sequence_Length)
    
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == Sequence_Length:
            Sequential_data.append([np.array(prev_days), i[-1]])

    random.shuffle(Sequential_data)

    buys = []
    sells = []
    
    for seq, target in Sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])

    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    Sequential_data = buys + sells
    
    random.shuffle(Sequential_data)

    X = []
    y = []
    
    for seq, target in Sequential_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X), y

In [10]:
main_df.head

<bound method NDFrame.head of             BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
1528968960    6480.000000        1.490900      96.519997       16.991997   
...                   ...             ...            ...             ...   
1534921800    6686.250000        0.478039      57.509998       18.782650   
1534921860    6686.250000        0.440793      57.500000        8.449425   
1534921920    6686.250000        2.678847      57.509998        6.070000   
1534921980    6686.250000        0.220156      57.509998       15.697691   
1534922040    6685.000000        6.401611      57.509998  

In [11]:
x_train, y_train = Preprocessing_df(main_df)
x_validation, y_validation = Preprocessing_df(validation_main_df)

In [12]:
print(f"x_train : {x_train.shape}")
print(f"x_validation : {x_validation.shape}")

x_train : (77920, 60, 9)
x_validation : (3860, 60, 9)


In [13]:
print(f"Train data : {len(x_train)} \nValidation data : {len(x_validation)}")
print(f"Dont buys : {y_train.count(0)}, buys : {y_train.count(1)}")
print(f"Validation Dont buys : {y_validation.count(0)}, buys : {y_validation.count(1)}")

Train data : 77920 
Validation data : 3860
Dont buys : 38960, buys : 38960
Validation Dont buys : 1930, buys : 1930


In [14]:
import time

Epochs = 20
Batch_size = 64

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization

In [16]:
model = Sequential()

model.add(LSTM(128, input_shape= (x_train.shape[1:]), return_sequences= True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences= True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation= "relu"))
model.add(Dropout(0.2))

model.add(Dense(2, activation= "softmax"))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 128)           70656     
                                                                 
 dropout (Dropout)           (None, 60, 128)           0         
                                                                 
 batch_normalization (Batch  (None, 60, 128)           512       
 Normalization)                                                  
                                                                 
 lstm_1 (LSTM)               (None, 60, 128)           131584    
                                                                 
 dropout_1 (Dropout)         (None, 60, 128)           0         
                                                                 
 batch_normalization_1 (Bat  (None, 60, 128)           512       
 chNormalization)                                       

In [18]:
optimization = tf.keras.optimizers.Adam(lr= 0.001)

In [19]:
model.compile(loss="sparse_categorical_crossentropy", optimizer= optimization, metrics= ["accuracy"])

In [ ]:
history = model.fit(x_train, np.array(y_train), batch_size=Batch_size, epochs= Epochs, validation_data= (x_validation, np.array(y_validation)))

Epoch 1/20
 846/1218 [===================>..........] - ETA: 28:05 - loss: 0.2393 - accuracy: 0.8893

In [ ]:
score = model.evaluate(x_validation, np.array(y_validation), verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])